# Chapter 1: Basic Prompt Structure

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [ ]:
!pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt}
        ]
    )
    return message.content[0].text

## 課程

Anthropic 提供兩種 API，分別是舊版的 [文字補全 API](https://docs.anthropic.com/claude/reference/complete_post) 和目前使用的 [Messages API](https://docs.anthropic.com/claude/reference/messages_post)。在本教學中，我們將專門使用 Messages API。

### 基本參數

至少，需要以下參數來調用 Claude 的 Messages API：

- `model`：你想要調用的 [API 模型名稱](https://docs.anthropic.com/claude/docs/models-overview#model-recommendations)。

- `max_tokens`：生成的最大 token 數量，Claude 可能在達到此上限前就停止。此參數僅指定絕對的最大 token 數量，並且這是一個 *硬性* 限制，可能導致 Claude 在單字或句子中途停止。

- `messages`：輸入訊息的陣列。我們的模型訓練方式是基於 `user` 和 `assistant` 之間的對話回合交替進行。在建立新的 `Message` 時，你需要使用 `messages` 參數來提供前面的對話內容，模型將基於這些內容生成下一個 `Message`。
  - 每條輸入訊息必須是一個包含 `role` 和 `content` 的物件。你可以只提供一條 `user` 角色的訊息，或是包含多條 `user` 和 `assistant` 訊息（這些訊息必須交替）。第一條訊息必須是 `user` 角色。

### 選用參數

除了基本參數，還有一些可選參數，例如：

- `system`：系統提示詞（詳見下文）。

- `temperature`：控制 Claude 回應的隨機性。在本課程與練習中，`temperature` 設定為 0。

若要查看所有 API 參數的完整列表，請參閱我們的 [API 文件](https://docs.anthropic.com/claude/reference/messages_post)。

### Examples

Let's take a look at how Claude responds to some correctly-formatted prompts. For each of the following cells, run the cell (`shift+enter`), and Claude's response will appear below the block.

In [ ]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

現在讓我們來看看一些沒有正確使用 Messages API 格式的提示詞。對於這些格式錯誤的提示詞，Messages API 會返回錯誤。

首先，我們有一個 Messages API 呼叫範例，其中 `messages` 陣列缺少 `role` 和 `content` 欄位。

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Hi Claude, how are you?"}
        ]
    )

# Print Claude's response
print(response[0].text)

Here's a prompt that fails to alternate between the `user` and `assistant` roles.

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ]
    )

# Print Claude's response
print(response[0].text)

`user` 和 `assistant` 訊息**必須交替出現**，並且訊息**必須以 `user` 回合開始**。在一個提示中，你可以包含多個 `user` 和 `assistant` 配對（類似於模擬多輪對話）。此外，你也可以在最終的 `assistant` 訊息中輸入部分文字，讓 Claude 繼續從該處接續（我們會在後續章節中詳細討論）。

#### 系統提示詞（System Prompts）

你也可以使用**系統提示詞**（System Prompt）。系統提示詞用於**向 Claude 提供上下文、指示和指導**，然後再讓它回答問題或執行任務。

從結構上來看，系統提示詞是獨立於 `user` 和 `assistant` 訊息列表的，因此它應該放在一個獨立的 `system` 參數中（可以參考筆記本 [Setup](#setup) 章節中的 `get_completion` 輔助函式結構）。

在本教學中，當我們需要使用系統提示詞時，已經在你的補全函式中提供了 `system` 欄位。如果你不想使用系統提示詞，只需將 `SYSTEM_PROMPT` 變數設為空字串即可。

#### System Prompt Example

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

### 為什麼要使用系統提示詞？

一個**撰寫良好的系統提示詞可以顯著提升 Claude 的表現**，例如增強 Claude 遵循規則與指示的能力。若想深入了解，請參閱我們的文件：[如何使用系統提示詞](https://docs.anthropic.com/claude/docs/how-to-use-system-prompts)。

### 進入練習

現在，我們將開始一些練習。如果你希望在不更改上述內容的情況下實驗課程中的提示詞，請滾動至課程筆記本的最底部，前往 [**範例遊樂場**](#example-playground)。

---

## Exercises
- [Exercise 1.1 - Counting to Three](#exercise-11---counting-to-three)
- [Exercise 1.2 - System Prompt](#exercise-12---system-prompt)

### 練習 1.1 - 數到三

請使用正確的 `user` / `assistant` 格式，修改以下 `PROMPT` 來讓 Claude **數到三**。輸出結果將顯示你的解法是否正確。

In [ ]:
# Prompt - this is the only field you should change
PROMPT = "[Replace this text]"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ If you want a hint, run the cell below!

In [ ]:
from hints import exercise_1_1_hint; print(exercise_1_1_hint)

### Exercise 1.2 - System Prompt

Modify the `SYSTEM_PROMPT` to make Claude respond like it's a 3 year old child.

In [ ]:
# System prompt - this is the only field you should change
SYSTEM_PROMPT = "[Replace this text]"

# Prompt
PROMPT = "How big is the sky?"

# Get Claude's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search(r"giggles", text) or re.search(r"soo", text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ If you want a hint, run the cell below!

In [ ]:
from hints import exercise_1_2_hint; print(exercise_1_2_hint)

### Congrats!

If you've solved all exercises up until this point, you're ready to move to the next chapter. Happy prompting!

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [ ]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"Hi Claude, how are you?"}
        ]
    )

# Print Claude's response
print(response[0].text)

In [ ]:
# Get Claude's response
response = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        messages=[
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ]
    )

# Print Claude's response
print(response[0].text)

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))